In [ ]:
!pip install kaggle albumentations opencv-python-headless torch torchvision pandas tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -q mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.6/752.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.5 MB/s eta 0:00:00


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# PlantVillage
!kaggle datasets download -d emmarex/plantdisease -p /content/data --unzip

# New Plant Diseases
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset -p /content/data --unzip

# Plant Disease Recognition
!kaggle datasets download -d rashikrahmanpritom/plant-disease-recognition-dataset -p /content/data --unzip


Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 83% 543M/658M [00:12<00:03, 30.1MB/s]
100% 658M/658M [00:12<00:00, 56.6MB/s]
Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
 98% 2.66G/2.70G [00:30<00:00, 62.3MB/s]
100% 2.70G/2.70G [00:30<00:00, 93.8MB/s]
Dataset URL: https://www.kaggle.com/datasets/rashikrahmanpritom/plant-disease-recognition-dataset
License(s): CC0-1.0
 97% 1.22G/1.25G [00:04<00:00, 233MB/s]
100% 1.25G/1.25G [00:04<00:00, 289MB/s]


In [ ]:
import os, shutil, random
from pathlib import Path
import cv2
import albumentations as A
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

RAW_DIR = Path("/content/plantdoctor/data/raw")
PROCESSED_DIR = Path("/content/plantdoctor/data/processed")
for folder in [RAW_DIR, PROCESSED_DIR]:
    shutil.rmtree(folder, ignore_errors=True)
    folder.mkdir(parents=True, exist_ok=True)

datasets = [
    (Path("/content/data/PlantVillage"), "pv"),
    (Path("/content/data/NewPlantDiseasesDataset"), "npd"),
    (Path("/content/data/PlantDiseaseRecognitionDataset"), "pdr")
]

for dataset_path, prefix in datasets:
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.jpg','.png','.jpeg')):
                species_name = Path(root).name.lower().replace(" ", "_")
                species_name = f"{prefix}_{species_name}"
                dest_dir = RAW_DIR / species_name
                dest_dir.mkdir(parents=True, exist_ok=True)
                shutil.copy(Path(root)/file, dest_dir / f"{prefix}_{file}")

print("✅ All datasets merged!")

#  Preprocessing + augmentation
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5)
])

TRAIN_DIR = PROCESSED_DIR / "train"
VAL_DIR   = PROCESSED_DIR / "val"
TEST_DIR  = PROCESSED_DIR / "test"
for folder in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

def preprocess_and_save(src_path, dest_path):
    img = cv2.imread(str(src_path))
    if img is None: return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = transform(image=img)["image"]
    dest_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(dest_path), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

# Split and parallel preprocess
all_tasks = []
for species in RAW_DIR.iterdir():
    if species.is_dir():
        images = list(species.glob("*.*"))
        random.shuffle(images)
        n = len(images)
        n_train = int(0.8*n)
        n_val   = int(0.1*n)
        n_test  = n - n_train - n_val
        for i, img_path in enumerate(images):
            if i < n_train:
                dest_file = TRAIN_DIR / species.name / img_path.name
            elif i < n_train+n_val:
                dest_file = VAL_DIR / species.name / img_path.name
            else:
                dest_file = TEST_DIR / species.name / img_path.name
            all_tasks.append((img_path, dest_file))

with ThreadPoolExecutor(max_workers=16) as executor:
    list(tqdm(executor.map(lambda x: preprocess_and_save(*x), all_tasks), total=len(all_tasks)))

print("✅ Preprocessing + augmentation complete!")

# Metadata table
import pandas as pd
metadata = []
for split, folder in [("train", TRAIN_DIR), ("val", VAL_DIR), ("test", TEST_DIR)]:
    for species in folder.iterdir():
        if species.is_dir():
            for img in species.glob("*.*"):
                metadata.append((str(img), species.name, split))

metadata_df = pd.DataFrame(metadata, columns=["image_path", "species", "split"])
metadata_df.head()


✅ All datasets merged!


100%|██████████| 20638/20638 [00:42<00:00, 481.13it/s]


✅ Preprocessing + augmentation complete!


,image_path,species,split
0,/content/plantdoctor/data/processed/train/pv_t...,pv_tomato_healthy,train
1,/content/plantdoctor/data/processed/train/pv_t...,pv_tomato_healthy,train
2,/content/plantdoctor/data/processed/train/pv_t...,pv_tomato_healthy,train
3,/content/plantdoctor/data/processed/train/pv_t...,pv_tomato_healthy,train
4,/content/plantdoctor/data/processed/train/pv_t...,pv_tomato_healthy,train


In [ ]:
import os
import mlflow

MLFLOW_DIR = "/content/drive/MyDrive/mlruns"
os.makedirs(MLFLOW_DIR, exist_ok=True)

# MLflow
mlflow.set_tracking_uri(f"file://{MLFLOW_DIR}")
mlflow.set_experiment("PlantDoctor_Experiment")

print("✅ MLflow tracking ready:", mlflow.get_tracking_uri())


✅ MLflow tracking ready: file:///content/drive/MyDrive/mlruns


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

class PlantDoctorDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.species2idx = {s: i for i, s in enumerate(sorted(df['species'].unique()))}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['image_path']).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.species2idx[row['species']]
        return {"image": img, "label": label, "species_name": row['species']}

train_df = metadata_df[metadata_df.split=="train"]
val_df   = metadata_df[metadata_df.split=="val"]
test_df  = metadata_df[metadata_df.split=="test"]

train_dataset = PlantDoctorDataset(train_df, transform=train_transform)
val_dataset   = PlantDoctorDataset(val_df, transform=val_transform)
test_dataset  = PlantDoctorDataset(test_df, transform=val_transform)

print(" Datasets ready :")
print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))
print("Test samples:", len(test_dataset))


✅ Datasets ready!
Train samples: 16504
Val samples: 2058
Test samples: 2076


In [ ]:
import shutil
import os


source_dir = '/content/plantdoctor'
output_zip = '/content/plantdoctor'


shutil.make_archive(output_zip, 'zip', source_dir)

print(f"✓ Successfully created zip file: {output_zip}.zip")
print(f"✓ Size: {os.path.getsize(output_zip + '.zip') / (1024*1024):.2f} MB")

✓ Successfully created zip file: /content/plantdoctor.zip
✓ Size: 836.94 MB


In [ ]:
import zipfile, os, shutil, re

src_zip = "plantdoctor.zip"
extract_dir = "plantdoctor_tmp1"
clean_zip = "plantdoctor_cleaned1.zip"

if os.path.exists(extract_dir):
    shutil.rmtree(extract_dir)

def clean_filename(name: str):
    parts = [p.strip() for p in name.split("/")]
    clean_name = "/".join(parts)
    # Remove space(s) before extension (like " Day 12 .jpg" → " Day 12.jpg")
    clean_name = re.sub(r"\s+\.(jpg|jpeg|png|JPG|JPEG|PNG)$", r".\1", clean_name)
    return clean_name

# 1️⃣ Extract and clean filenames
with zipfile.ZipFile(src_zip, 'r') as z:
    for member in z.namelist():
        fixed_name = clean_filename(member)
        target_path = os.path.join(extract_dir, fixed_name)

        if member.endswith('/'):
            os.makedirs(target_path, exist_ok=True)
            continue

        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        with z.open(member) as src, open(target_path, "wb") as dst:
            dst.write(src.read())

print("✅ Extraction + cleaning done")

#  Create the cleaned zip
with zipfile.ZipFile(clean_zip, "w", zipfile.ZIP_DEFLATED) as z:
    for root, _, files in os.walk(extract_dir):
        for file in files:
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, extract_dir)
            z.write(full_path, rel_path)

print(f"✅ New cleaned ZIP created: {clean_zip}")

#  Verify that there are no spaces before extensions or trailing spaces
bad_files = []
pattern = re.compile(r"\s+\.(jpg|jpeg|png|JPG|JPEG|PNG)$")
with zipfile.ZipFile(clean_zip, "r") as z:
    for name in z.namelist():
        if name != name.strip() or pattern.search(name):
            bad_files.append(name)

if bad_files:
    print("⚠️ Files still problematic:")
    for f in bad_files:
        print(f" - {f}")
else:
    print("🎉 Verification passed — all filenames cleaned successfully.")


✅ Extraction + cleaning done
✅ New cleaned ZIP created: plantdoctor_cleaned1.zip
🎉 Verification passed — all filenames cleaned successfully.
